# Verify one claim

In [12]:
import os
os.chdir('/Users/dianaavalos/PycharmProjects/InternationalBridgesToJustice')
import json
from src.internationalbridgestojustice.openai_utils import openai_client
from src.internationalbridgestojustice.chromadb_utils import (
    load_collection,
    perform_similarity_search_in_collection,
)
from src.internationalbridgestojustice.file_manager import (
    load_legal_chunks,
    build_context_string_from_retrieve_documents,
)
from src.internationalbridgestojustice.openai_utils import (get_openai_response,openai_client)
from src.internationalbridgestojustice.query_functions import (
    format_prompt_for_claim_verification,
    retrieve_source_titles_from_chunks,
)
from src.internationalbridgestojustice.config import Paths

In [14]:
with open(Paths.PATH_FILE_PROMPT_CLAIM_VERIFICATION, "r") as f:
    prompt_claim_verification = f.read()

print(prompt_claim_verification[:3000])  # Print the first 1000 characters of the prompt

## **Claim Verification Instructions**

You need to determine whether the database contains enough information to fact-check the claim, and then decide whether the claim is **Supported**, **Contradicted**, or **Inconclusive** based on that information. You can cite specific laws or legal chapters to justify your decision.

---

### Very important:
Make sure that the information used for verification comes from the **correct country**.  
You can find the country name in the `'metadata':'title'` or `'metadata':'country'` fields of the context.

---

### Is there enough information?

**Does the database provide enough information to fact-check the claim?**

- If **no**, label the claim as `###Inconclusive###`
- If **yes**, state the judgment as one of the following categories, marked with ###:

#### `###Supported###`
A claim is **supported** by the database if everything in the claim is supported and nothing is contradicted by the information in the database.  
There can be some results t

In [17]:
with open('data/prompts/system_prompt_claim_verification.md', "r") as f:
    system_prompt_claim_verification = f.read()

print(system_prompt_claim_verification)  

You are a helpful assistant who can judge whether a claim is supported or contradicted by the search results, 
or whether there is no enough information to make a judgement.


In [6]:
claim_to_verify = (
    "In Burundi, until proven innocent, the accused has to remain in prison."
)

collection = load_collection(
    chroma_collection_file_path=Paths.PATH_CHROMADB_v2,
    collection_name=Paths.COLLECTION_NAME_v2,
)
print(f"Collection contains {collection.count()} documents.")


/Users/dianaavalos/PycharmProjects/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Collection contains 1010 documents.


In [7]:
results = perform_similarity_search_in_collection(
    collection=collection,
    query_text=claim_to_verify,
    metadata_param="country",
    metadata_value="Burundi",
    number_of_results_to_retrieve=5,
)
print(results)

{'ids': [['fbb5f13cdbe2bffff045f9d9ee8f52fbd36312466e61e5dbf9c038ac6b1de857', '4055b0b548981ef6bc0365fdf9604729a9234850ac5f2af42c8442578a2a6baa', '3a9a6a2ba8bd03e2671881b29c89942f5ce9dbdac556d5cced394c3822351623', '4099a2b9bddbcdd9290c4be1531fd2395ec264fae1bc1e6831d252b99d4fb2db', '8b72dd09364dd9da460e61e8dac296adb421321f0b26dd8b2e9d1824e9089f60']], 'embeddings': None, 'documents': [['However, pending a conviction judgment being rendered res judicata, the presumption of innocence remains in favor of the accused even if in the meantime the latter remains in pre-trial detention.\n\nThus, while inviting all actors in the Burundian criminal justice system to respect and enforce, as much as possible, the provisions of the C.P.P, we request that public authorities make available all necessary means to enable the effective implementation of the C.P.P so that the respect for the rights of the person accused of crimes remains a fundamental principle to which Burundian legislation attaches param

In [9]:
res_summary = [r["title_bis"] for r in results["metadatas"][0]]
print(f"Titles of chunks selected: {res_summary}")

Titles of chunks selected: ['defensewiki_doc.Manuel_sur_les_procedures_darrestation_et_de_detention_instituees_par_le_Code_de_Procedure_Penale_du_Burundi.Toutefois, en attendant qu’un jugement de condamnation.0', 'defensewiki_doc.Petit_Manuel_de_Defense_Penale_du_Burundi_-_Principes_Fondamentaux.La présomption d’innocence.0', 'defensewiki_doc.Burundi-fr.Les droits de l’accusé et du détenu.0', 'Burundi.satisfaire aux conditions de l’article 154 alinéa.0', 'defensewiki_doc.Petit_Manuel_de_Defense_Penale_du_Burundi_-_Strategies_de_Defense.Culpabilité admise ou contestée.2']


In [15]:
context_text = build_context_string_from_retrieve_documents(results)

prompt_claim_verification_Burundi = format_prompt_for_claim_verification(
    prompt_claim_verification, claim=claim_to_verify, context=context_text
)

In [19]:
schema_claim_verification = {
    "type": "object",
    "name": "ClaimVerificationResponse",
    "description": "Response to a claim verification request",
    "properties": {
         "claim": {
            "type": "string",
            "description": "The initial claim.",
        },
        "decision": {
            "type": "string",
            "enum": [
                "Supported",
                "Contradicted",
                "Inconclusive",
            ],
            "description": "The claim verification decision.",
        },
        "full_answer": {
            "type": "string",
            "description": "Specific laws or legal chapters to justify your decision if applicable (not necessary for Inconclusive).",
        },
        "sources": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Chunk titles that support the claim.",
        },
        "document_ids": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Chunk hashes that support the claim.",
        },
        "distances": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Distances of the chunks that support the claim.",
        },
    },
    "required": [
        "claim",
        "decision",
        "sources",
        "document_ids",
        "distances",
    ],
}

In [23]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "ClaimVerificationResponse",
        "schema": schema_claim_verification,
    },
}
type(response_format)

dict

In [24]:
answer = get_openai_response(client=openai_client, categorize_system_prompt=system_prompt_claim_verification, prompt=prompt_claim_verification_Burundi, response_format=response_format, model="gpt-4o-mini", temperature=0.1)
print("\nOpenAI response:\n", answer)


OpenAI response:
 {"claim":"In Burundi, until proven innocent, the accused has to remain in prison.","decision":"###Contradicted###","full_answer":"The presumption of innocence is a fundamental principle in Burundi, as stated in Article 40 of the Constitution, which affirms that everyone accused of a criminal act is presumed innocent until their guilt has been legally established. While pre-trial detention can occur, it is not a requirement that the accused must remain in prison until proven innocent. The context indicates that the presumption of innocence remains in favor of the accused, even if they are in pre-trial detention, which contradicts the claim that they must remain in prison until proven innocent.","sources":["The Presumption of Innocence","The Rights of the Accused and the Detained"],"document_ids":["7497","7001"],"distances":["0","0"]}


In [25]:
with open(
    "data/verified_claims/claims_1_Burundi.jsonl", "a", encoding="utf-8"
) as jsonl_file:
    jsonl_file.write(json.dumps(answer) + "\n")